In [1]:
import gc
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

In [2]:
# ----- Функции подготовки таблицы ----------------

In [3]:
def groupType(x):
    res = list(set(x))
    return sum(res) if len(res) > 0 else -1
# -----------------------------------------------------------
def groupLen(x):
    return len(list(set(x)))
# -----------------------------------------------------------
def dropCopies(df, column='labels'):
    _ = df.copy()
    _[column] = _[column].apply(lambda x: list(dict.fromkeys(x)))
    return _
# -----------------------------------------------------------
def trimLabels(df, count=20, column='labels'):
    _ = df.copy()
    _[column] = _[column].apply(lambda x: x[:count])
    return _
# -----------------------------------------------------------
def cloneEvents(df):
    result = df.copy()
    result['type'] = 0
    _ = df.copy()
    _['type'] = 1
    result = pd.concat([result, _])
    _ = df.copy()
    _['type'] = 2
    result = pd.concat([result, _])
    return result
# -----------------------------------------------------------
def fillEvents(df, clone=True, value=[-1]):
    _ = df[['session','type']].groupby(['session'])\
            .agg(ctype = ('type', groupType), count = ('type', groupLen))
    result = pd.DataFrame()
    tmp = df[df['session'].isin(_[_['ctype'] == 0].index)].copy()
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 1
    result = pd.concat([result, tmp])
    tmp['type'] = 2
    result = pd.concat([result, tmp])

    tmp = df[df['session'].isin(_[_['ctype'] == 1].index)].copy().drop_duplicates(['session'])
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 2
    result = pd.concat([result, tmp])

    tmp = df[df['session'].isin(_[_['ctype'] == 2].index)].copy().drop_duplicates(['session'])
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 1
    result = pd.concat([result, tmp])
    
    tmp = df[df['session'].isin(_[(_['ctype'].isin([1, 2]))&(_['count'] == 1)].index)].copy()
    if not clone: tmp['aid'] = tmp['aid'].apply(lambda x: value)
    tmp['type'] = 0
    result = pd.concat([result, tmp])

    tmp = df[df['session'].isin(_[(_['ctype'] == 3)&(_['count'] == 2)].index)].copy()\
                .drop_duplicates(['session'])
    if not clone: tmp['aid'].apply(lambda x: [])
    tmp['type'] = 0
    result = pd.concat([result, tmp])
    return result

In [4]:
# ----- Функции загрузки данных -------------------

In [5]:
# ==================================================================
# ----- Загрузка датасета ------------------------------------------
# ==================================================================
def sortAndDrop(df, columns, asc=True, drop = [], time_elapsed = None):
    ''' Сортируем по убыванию и удаляем столбцы '''
    df.sort_values(columns, ascending=asc, inplace=True)
    if time_elapsed is not None:
        df = df.merge(df.groupby(['session'])['ts'].max().reset_index(), 
                      how='left', on='session').rename(columns={'ts_x': 'ts', 'ts_y': 'max_ts'})
        df = df.loc[df['ts'] > df['max_ts'] - time_elapsed]
    if len(drop) > 0:
        df.drop(drop, inplace=True, axis=1)
    return df
# -----------------------------------------------------------------
def loadDataset(path, time_elapsed = None):
    ''' Загрузка датасета с предобработкой '''
    df = pd.read_parquet(path)
    return sortAndDrop(df, ['session', 'ts'], [True, False], [], time_elapsed)

In [6]:
# ----- Функции вычисления истории ----------------

In [7]:
# ==================================================================
# ----- Функции получения истории без учёта типов ------------------
# ==================================================================
def getLastNoType(df, count=20, order=[1, 0 ,2], fill=True):
    def sort(x):
        return x.map(lambda x: order.index(x))
    # Берём отсортированные по убыванию времени товары
    # группируем по сессии и типам и выбираем N товаров
    _ = df.copy().groupby(['session', 'type']).agg(lambda x: list(dict.fromkeys(x))).reset_index()
    if type(count) is int:
        count = {0: count, 1:count, 2:count}
    elif type(count) is list:
        count = {0: count[0], 1:count[1], 2:count[2]}
    for ind in count.keys():
        _.loc[_['type'] == ind, 'aid'] = _[_['type'] == ind]['aid'].apply(lambda x: x[:count.get(ind)])
    # если указан порядок - сортируем
    if order is not None and type(order) is list:
        _.sort_values(by='type', inplace=True, key=sort)
    _ = _.groupby('session').agg(labels = ('aid', sum)).reset_index()
    _ = dropCopies(_)
    # копируем для всех типов
    return cloneEvents(_) if fill else _
# ------------------------------------------------------------------
def getLastNoOrder(df, count=20, order=None):
    return getLastNoType(df, count, None)
# ------------------------------------------------------------------
def getLastNoTypeNoCopies(df, count=20, order=[1, 0 ,2], fill=True):
    def sort(x):
        return x.map(lambda x: order.index(x))
    # Берём отсортированные по убыванию времени товары
    # группируем по сессии и типам и выбираем N товаров
    _ = df.copy().groupby(['session', 'type']).agg(lambda x: list(dict.fromkeys(x))).reset_index()
    # если указан порядок - сортируем
    if order is not None and type(order) is list:
        _.sort_values(by='type', inplace=True, key=sort)
    _ = _.groupby('session').agg(labels = ('aid', sum)).reset_index()
    _ = dropCopies(_)
    _ = trimLabels(_, count)
    # копируем для всех типов
    return cloneEvents(_) if fill else _
# ------------------------------------------------------------------
def getLastNoTypeOnPercent(df, count=0.5, order=[1, 0 ,2], fill=True):
    def sort(x):
        return x.map(lambda x: order.index(x))
    def apply(x):
        rlen = round(len(x) * count)
        return x[:rlen if rlen > 0 else 1]
    # Берём отсортированные по убыванию времени товары
    # группируем по сессии и типам и выбираем P товаров в зависимости от длинны сессии
    _ = df.copy().groupby(['session', 'type']).agg(lambda x: list(dict.fromkeys(x))).reset_index()
    _['aid'] = _['aid'].apply(apply)
    # если указан порядок - сортируем
    if order is not None and type(order) is list:
        _.sort_values(by='type', inplace=True, key=sort)
    _ = _.groupby('session').agg(labels = ('aid', sum)).reset_index()
    _ = dropCopies(_)
    # копируем для всех типов
    return cloneEvents(_) if fill else _
# ==================================================================
# ----- Функции получения истории с учётом типов -------------------
# ==================================================================
def getLastWithType(df, count=20, order=None, clone=False):
    # Берём отсортированные по убыванию времени товары
    # группируем по сессии и типам и выбираем N товаров
    _ = df.copy().groupby(['session', 'type']).agg(lambda x: list(dict.fromkeys(x))[:count]).reset_index()
    __ = pd.DataFrame()
    # Если указан порядок
    if order is not None:
        # Если порядок - число, значит мы хотим определённый тип раскидать на все
        if type(order) is int:
            _ = _.groupby(['session', 'type']).agg(aid = ('aid', sum)).reset_index()
            t = cloneEvents(_[_['type'] == order])
            _ = pd.concat([t, _[~(_['session'].isin(t['session']))]])
            _['type'] = _['type'].astype('uint8')
        # Если порядок - словарь, значит мы меняем тип предсказания. 
        # Например, 0 предсказываем по 1
        elif type(order) is dict:
            _ = _[_['type'].isin(order.keys())].map(order)
    # Заполняем недостающие сессии
    __ = fillEvents(_, clone=clone)
    return pd.concat([_, __]).rename(columns = {'aid':'labels'})
# ----------------------------------------------------------------
def getLastWithTypeClone(df, count=20, order=None):
    return getLastWithType(df, count, order, clone=True)
# ----------------------------------------------------------------
def getLastWithTypeOrder(df, count=20, order=[0, 0, 1]):
    return getLastWithType(df, count, order)
# ----------------------------------------------------------------
def getLastWithTypeHardOrder(df, count={0:5, 1:[10,5,5], 2:[20,20,5]}, order=[1, 0, 2]):
    ''' Сложное предсказание 
        Для каждого типа выбираем N товаров других типов '''
    # Берём отсортированные по убыванию времени товары
    # группируем по сессии и типам
    _ = pd.DataFrame()
    for ind in count.keys():
        res = getLastNoType(df, count.get(ind), order, fill=False)
        res['type'] = ind
        _ = pd.concat([_, res])
    # Заполняем недостающие сессии
    _.rename(columns = {'labels':'aid'}, inplace=True)
    __ = fillEvents(_, clone=False)
    return pd.concat([_, __]).rename(columns = {'aid':'labels'})

In [8]:
# ----- Функции вычисления популярных товаров -----

In [9]:
# ==================================================================
# ----- Функции получения популярных файлов ------------------------
# ==================================================================
def getPopularityWithType(df, count=20):
    ''' Получить популярные товары с учётом типов событий '''
    # группируем товары по aid и типу, получаем количество 
    # и сортируем по типу и сессии(количеству) в обратном порядке
    result = df[['aid', 'type', 'session']].groupby(['aid', 'type'])\
            .count().sort_values(['type', 'session'], ascending=False).reset_index()
    # объединяем по N последних в таблицу
    result = pd.concat([result.loc[result['type'] == 0].iloc[:count], \
                    result.loc[result['type'] == 1].iloc[:count], \
                    result.loc[result['type'] == 2].iloc[:count]])
    # группируем по типам
    result = result[['aid', 'type']].groupby('type')\
                .agg(lambda x: list(dict.fromkeys(x))).reset_index()
    result.columns = ['type', 'labels']
    return result
# -----------------------------------------------------------------
def getPopularityNoType(df, count=20):
    ''' Получить популярные товары без учёта типов событий '''
    # группируем товары по aid, получаем количество 
    # и сортируем по сессии(количеству) в обратном порядке
    result = df[['aid', 'session']].groupby(['aid'])\
            .count().sort_values(['session'], ascending=False).reset_index()
    # объединяем по N последних в таблицу
    result = result.iloc[:count]
    result['type'] = 0
    # группируем по типам
    result = result[['aid', 'type']].groupby('type')\
                .agg(lambda x: list(dict.fromkeys(x))).reset_index()
    result.drop('type', axis=1, inplace=True)
    result = pd.concat([result, result, result], ignore_index=True).reset_index()
    result.columns = ['type','labels']
    return result

In [10]:
# ----- Функции расчёта одиночных товаров -----

In [11]:
def getSingleItems(df, time_elapsed=7*24*60*60):
    _ = df.copy().groupby(['session', 'aid']).agg(count=('ts', 'count')).reset_index()
    drop = _[_['count'] > 1]['aid'].unique()
    _ = _[(~_['aid'].isin(drop))&(_['count'] == 1)]['aid'].unique()
    return _

In [12]:
# ==================================================================
# ----- Конфигурация сессии для экспериментов-----------------------
# ==================================================================
LOCAL = True  # Тип метрики. True - локально, False - Kaggle
SAVE  = True # Сохранять ли файлы. Нужно при сохранении датасета
SHOW  = True # Выводить ли таблицы в процессе
# -----------------------------------------------------------------
params = {
    'use_popular': False, # Использовать ли популярные файлы при засылке
    'drop_single': False, # Убирать ли товары, покупаемые только раз
    #'time': 2*24*60*60,
    'last': {
        'func': getLastNoType,
        'params': {
            #'count': {0:[5, 0, 0], 1:[15,5,0], 2:[15,15,0]},
            'count': 40,
            'order': [1, 0, 2],
        },
        'filename': 'lastItems'
    },
    'popular': {
        'func': getPopularityWithType,
        'params': {
            'count': 40
        },
        'filename': 'popular'
    }
}

In [13]:
# ----- Загружаем данные ------------------------------------

In [14]:
%%time
# ~ 10s
test  = loadDataset('../input/otto-analyse-data/test.parquet', time_elapsed=params.get('time'))
local = loadDataset('../input/otto-train-and-test-data-for-local-validation/test.parquet', time_elapsed=params.get('time'))
if SHOW: display(test.head(4))

,session,aid,ts,type
0,12899779,59625,1661724000,0
5,12899780,1142000,1661724155,0
4,12899780,736515,1661724136,0
3,12899780,973453,1661724109,0


CPU times: user 7.59 s, sys: 2.13 s, total: 9.72 s
Wall time: 10 s


In [15]:
# ----- Вычисляем одиночные товары --------------------------

In [16]:
%%time
# ~ 2m 25s
singleItems = None
if params['drop_single']:
    train = pd.read_parquet('../input/otto-analyse-data/train_no_duplicates.parquet')
    singleItems = getSingleItems(train)
    del train; gc.collect()
    test  = test[~test['aid'].isin(singleItems)]
    local = local[~local['aid'].isin(singleItems)]
    if SHOW: display(test)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


In [17]:
# ----- Вычисляем историю ----------------------------------

In [18]:
%%time
# ~ 1m 40s
tLast = params['last']['func'](test, **params['last']['params'])
lLast = params['last']['func'](local, **params['last']['params'])
if SAVE:
    tLast.astype({'session': 'uint32', 'type':'uint8'}).to_parquet(f'test_{params["last"]["filename"]}.parquet', index=False)
    lLast.astype({'session': 'uint32', 'type':'uint8'}).to_parquet(f'local_{params["last"]["filename"]}.parquet', index=False)
if SHOW: display(tLast.head(4))

,session,labels,type
0,12899779,[59625],0
1,12899780,"[1142000, 736515, 973453, 582732]",0
2,12899781,"[199008, 918667, 194067, 57315, 141736]",0
3,12899782,"[834354, 740494, 987399, 889671, 127404, 17111...",0


CPU times: user 1min 37s, sys: 3.87 s, total: 1min 41s
Wall time: 1min 41s


In [19]:
#_ = tLast.explode('labels').groupby('session').agg(count=('labels', lambda x: len(dict.fromkeys(x))))
#_.loc[_['count'] >= 20].shape[0] / _.shape[0]

In [20]:
# ----- Вычисляем популярные товары -------------------------

In [21]:
%%time
# ~ 4s
tPopular = params['popular']['func'](test, **params['popular']['params'])
lPopular = params['popular']['func'](local, **params['popular']['params'])
if SAVE:
    tPopular.astype({'type':'uint8'}).to_parquet(f'test_{params["popular"]["filename"]}.parquet', index=False)
    lPopular.astype({'type':'uint8'}).to_parquet(f'local_{params["popular"]["filename"]}.parquet', index=False)
if SHOW: display(tPopular)

,type,labels
0,0,"[1460571, 485256, 108125, 986164, 1551213, 754..."
1,1,"[485256, 33343, 1460571, 986164, 554660, 66065..."
2,2,"[986164, 1460571, 329725, 1043508, 332654, 688..."


CPU times: user 3.46 s, sys: 322 ms, total: 3.79 s
Wall time: 3.77 s


In [22]:
# ----- PREDICT -----

In [23]:
def localMetrics(sub):
    # -- load ground truth
    df_true = pd.read_parquet('../input/otto-train-and-test-data-for-local-validation/test_labels.parquet')
    df_true = df_true.rename(columns = {'ground_truth': 'aids'} )
    
    # -- calculate metrics
    lsub = sub.copy()
    lsub['session'] = lsub.session_type.apply(lambda x: np.int64(x.split('_')[0]))
    lsub['type']    = lsub.session_type.apply(lambda x: x.split('_')[1]) 
    lsub['labels']  = lsub.labels.apply(lambda x: [np.int64(i) for i in x.split()[:20]])

    test_labels = df_true.copy()
    test_labels = test_labels.merge(lsub, how='left', on=['session', 'type'])
    test_labels['labels'] = test_labels['labels'].fillna('')
    
    test_labels['hits']     = test_labels.apply(lambda df: len(set(df.aids).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.aids.str.len().clip(0,20)  
    
    recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 
    r0,r1,r2 = recall_per_type['clicks'],  recall_per_type['carts'],  recall_per_type['orders']
    
    print(f"{r0*0.1 + r1*0.3 + r2*0.6:.3f} = {r0*0.1:.3f} + {r1*0.3:.3f} + {r2*0.6:.3f}")
    score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
    print('score:', score)
    return score

In [24]:
submission = lLast if LOCAL else tLast
if SHOW: display(submission.head(4))

,session,labels,type
0,11098528,[11830],0
1,11098529,[1105029],0
2,11098530,"[409236, 264500]",0
3,11098531,"[1365569, 1728212, 1271998, 624163, 1553691, 3...",0


In [25]:
# ----- Добавляем популярные товары -------------------------------

In [26]:
%%time
# ~ 1m
if params['use_popular']:
    submission = submission.merge(lPopular if LOCAL else tPopular, on=['type'], how='left')
    submission['labels'] = submission['labels_x'] + submission['labels_y']
    submission.drop(['labels_x', 'labels_y'], axis=1, inplace=True)
    if SHOW: display(submission.head(4))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


In [27]:
# ----- Убираем одиночные товары ---------------------------------

In [28]:
if params['drop_single']:
    tmp = submission.explode('labels')
    tmp['labels'] = tmp['labels'].fillna(-1).map(lambda x: int(x))
    tmp['labels'] = tmp['labels'].astype('uint32')
    tmp['type'] = tmp['type'].astype('uint8')
    tmp = tmp[~tmp['labels'].isin(singleItems)]
    submission = tmp.groupby(['session', 'type']).agg(lambda x: list(dict.fromkeys(x))).reset_index()
    if SHOW: display(submission.head(4))

In [29]:
# ----- Подготавливаем предсказание --------------------------

In [30]:
%%time
# ~ 2m
submission['labels'] = submission['labels'].map(lambda x: ' '.join(str(i) for i in list(dict.fromkeys(x))))
submission['type'] = submission['type'].map({0: 'clicks', 1: 'carts', 2: 'orders'})
submission['session_type'] = submission['session'].astype('str') + '_' + submission['type']
submission.drop(['session', 'type'], inplace=True, axis=1)
if SAVE: submission.to_csv('submission.csv', index=False)
if SHOW: display(submission.head(4))

,labels,session_type
0,11830,11098528_clicks
1,1105029,11098529_clicks
2,409236 264500,11098530_clicks
3,1365569 1728212 1271998 624163 1553691 396199 ...,11098531_clicks


CPU times: user 26.6 s, sys: 1.77 s, total: 28.3 s
Wall time: 28.4 s


In [31]:
%%time
# ~ 2m 30s
if LOCAL: # 0.477 = 0.031 + 0.093 + 0.353 0.4770073009270803
          # 0.475 = 0.030 + 0.093 + 0.353 0.4754129683568511
    localMetrics(submission)

0.481 = 0.032 + 0.093 + 0.356
score: 0.48149111874677053
CPU times: user 1min 28s, sys: 5.36 s, total: 1min 33s
Wall time: 1min 33s


In [32]:
# --- Подготовка таблицы для обработки ---

In [33]:
%%time
# ~ 1m
if SAVE:
    test  = pd.read_parquet('../input/otto-analyse-data/test.parquet')
    local = pd.read_parquet('../input/otto-train-and-test-data-for-local-validation/test.parquet')
    test  = pd.concat([test, fillEvents(test)])
    local = pd.concat([local, fillEvents(local)])
    test['type'] = test['type'].astype('uint8')
    local['type'] = local['type'].astype('uint8')
    test.to_parquet('test_extended.parquet', index=False)
    local.to_parquet('local_extended.parquet', index=False)

CPU times: user 56.8 s, sys: 2.53 s, total: 59.3 s
Wall time: 58.6 s
